In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# dfの各列の型を設定しメモリ軽減
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train.head()

In [ ]:
train = reduce_mem_usage(train)
train.info()

In [ ]:
# dfをpickleで保存
train.to_pickle('/kaggle/working/JSMP_train.pickle')

### 説明変数について

In [ ]:
train = pd.read_pickle('/kaggle/working/JSMP_train.pickle')

In [ ]:
# データの分布
train['resp'].hist(bins=30)

左右はほぼで対称、０近辺にデータが集中している。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# リターンの累積和
train['resp'].cumsum().plot(label='resp')
train['resp_1'].cumsum().plot(label='resp_1')
train['resp_2'].cumsum().plot(label='resp_2')
train['resp_3'].cumsum().plot(label='resp_3')
train['resp_4'].cumsum().plot(label='resp_4')
plt.legend()

最初の50,000個のデータでは着々とリターンを積み上げており、その後はしばらく横ばいが続き、最後の50,000個でリターンが積みあがっている様子が見られる。最初の上昇が継続している期間は、上昇トレンドが発生している可能性がある。最初の期間の特徴をうまく抽出してあげることがパフォーマンスの良いモデルを構築するカギとなるのかもしれない。

In [ ]:
# 対数変換
np.log(train['resp']+1).hist(bins=30, color='lightblue', label='log(y)')
train['resp'].hist(bins=30, histtype='step', color='red', label='y')
plt.legend()

対数変換前に比べると、マイナスのばらつきが広がった。また、０近辺のデータが増えている。対数変換により大して変化は見られなかったが、目的変数に用いてみるのも良いかもしれない。ただし、その場合は、予測値を逆変換する必要がある。

In [ ]:
train['resp'].apply(lambda x:1 if x>0 else -1).value_counts()

In [ ]:
plt.pie(train['resp'].apply(lambda x:1 if x>0 else -1).value_counts(),
        labels=['plus', 'minus'], startangle=90, autopct='%1.1f%%')

若干だがプラスのデータの方が多かった。学習データに用いる場合は、ほぼ1:1なので問題はなさそう。

In [ ]:
train['resp'].describe()

In [ ]:
np.log(train['resp']+1).describe()

対数変換による効果はあまりなさそう。

In [ ]:
train['resp'].cumsum().plot(label='resp')
train['resp'].rolling(window=50).mean().cumsum().plot(label='MA_50')
train['resp'].rolling(window=50000).mean().cumsum().plot(label='MA_50000')
train['resp'].rolling(window=100000).mean().cumsum().plot(label='MA_100000')
plt.legend()

ウィンドウ幅として、50, 50000, 100000を指定してみた。結果としては、ウィンドウ幅による差はほとんど見られなかった。また、移動平均の累積和は右上がりとなっていた。50,000~150,000の期間のrespでは、横ばいの傾向が見られたが、移動平均は上昇傾向にある。

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
train['resp'].rolling(window=50).mean().hist(bins=30)
plt.title('window=50')

plt.subplot(1, 3, 2)
train['resp'].rolling(window=50000).mean().hist(bins=30)
plt.title('window=50000')

plt.subplot(1, 3, 3)
train['resp'].rolling(window=100000).mean().hist(bins=30)
plt.title('window=100000')

中心極限定理に基づくと正規分布に従うはずである。window=50は正規分布に近い形をしているが、他の分布に関しては複数の分布から構成されているように見える。ウィンドウの幅は関係ないようにも思えたが、ちゃんと選ぶ必要があるのかもしれない。